In [2]:
import pandas as pd
from src.data.mimic_iii.real_dataset import MIMIC3RealDataset

2024-01-06 14:11:28.545214: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
gsu_features_path = '/Users/jk1/temp/treatment_effects/preprocessing/gsu_Extraction_20220815_prepro_25112023_213851/preprocessed_features_25112023_213851.csv'
gsu_continuous_outcomes_path = '/Users/jk1/temp/treatment_effects/preprocessing/gsu_Extraction_20220815_prepro_25112023_213851/preprocessed_outcomes_continuous_25112023_213851.csv'

In [4]:
features_df = pd.read_csv(gsu_features_path)
outcomes_df = pd.read_csv(gsu_continuous_outcomes_path)

# Preprocessing

In [5]:
last_timestep = None

In [6]:
# Features data
features_df.drop(columns=['impute_missing_as'], inplace=True)
pivoted_features_df = features_df.pivot(index=['case_admission_id', 'relative_sample_date_hourly_cat'],
                                        columns='sample_label', values='value')

# get rid of multiindex
pivoted_features_df = pivoted_features_df.rename_axis(None, axis=1).reset_index()

if last_timestep is not None:
    pivoted_features_df = pivoted_features_df[
        pivoted_features_df.relative_sample_date_hourly_cat < last_timestep + 1]

# seperate out treatment features
treatment_df = pivoted_features_df[
    ['case_admission_id', 'relative_sample_date_hourly_cat', 'anti_hypertensive_strategy']]
pivoted_features_df.drop(columns=['anti_hypertensive_strategy'], inplace=True)

# Set the 2-level index:
treatment_df.set_index(keys=["case_admission_id", "relative_sample_date_hourly_cat"], drop=True, inplace=True)
pivoted_features_df.set_index(keys=["case_admission_id", "relative_sample_date_hourly_cat"], drop=True,
                              inplace=True)

# Outcome data
if last_timestep is not None:
    outcomes_df = outcomes_df[outcomes_df.relative_sample_date_hourly_cat < last_timestep + 1]
outcomes_df.set_index(keys=["case_admission_id", "relative_sample_date_hourly_cat"], drop=True, inplace=True)
reformatted_outcomes_df = outcomes_df[['nihss_delta_at_next_ts']]

# Load into Dataset

In [7]:
# rename index 'case_admission_id' to 'subject_id'
treatment_df.index = treatment_df.index.rename(['subject_id', 'relative_sample_date_hourly_cat'])
pivoted_features_df.index = pivoted_features_df.index.rename(['subject_id', 'relative_sample_date_hourly_cat'])
reformatted_outcomes_df.index = reformatted_outcomes_df.index.rename(['subject_id', 'relative_sample_date_hourly_cat'])

In [9]:
static_features = pivoted_features_df.copy()
static_features = static_features.drop(columns=pivoted_features_df.columns)

In [10]:
static_features

Empty DataFrame
Columns: []
Index: [(100023_4784, 0), (100023_4784, 1), (100023_4784, 2), (100023_4784, 3), (100023_4784, 4), (100023_4784, 5), (100023_4784, 6), (100023_4784, 7), (100023_4784, 8), (100023_4784, 9), (100023_4784, 10), (100023_4784, 11), (100023_4784, 12), (100023_4784, 13), (100023_4784, 14), (100023_4784, 15), (100023_4784, 16), (100023_4784, 17), (100023_4784, 18), (100023_4784, 19), (100023_4784, 20), (100023_4784, 21), (100023_4784, 22), (100023_4784, 23), (100023_4784, 24), (100023_4784, 25), (100023_4784, 26), (100023_4784, 27), (100023_4784, 28), (100023_4784, 29), (100023_4784, 30), (100023_4784, 31), (100023_4784, 32), (100023_4784, 33), (100023_4784, 34), (100023_4784, 35), (100023_4784, 36), (100023_4784, 37), (100023_4784, 38), (100023_4784, 39), (100023_4784, 40), (100023_4784, 41), (100023_4784, 42), (100023_4784, 43), (100023_4784, 44), (100023_4784, 45), (100023_4784, 46), (100023_4784, 47), (100023_4784, 48), (100023_4784, 49), (100023_4784, 50), (100023_4784, 51), (100023_4784, 52), (100023_4784, 53), (100023_4784, 54), (100023_4784, 55), (100023_4784, 56), (100023_4784, 57), (100023_4784, 58), (100023_4784, 59), (100023_4784, 60), (100023_4784, 61), (100023_4784, 62), (100023_4784, 63), (100023_4784, 64), (100023_4784, 65), (100023_4784, 66), (100023_4784, 67), (100023_4784, 68), (100023_4784, 69), (100023_4784, 70), (100023_4784, 71), (1002417_9090, 0), (1002417_9090, 1), (1002417_9090, 2), (1002417_9090, 3), (1002417_9090, 4), (1002417_9090, 5), (1002417_9090, 6), (1002417_9090, 7), (1002417_9090, 8), (1002417_9090, 9), (1002417_9090, 10), (1002417_9090, 11), (1002417_9090, 12), (1002417_9090, 13), (1002417_9090, 14), (1002417_9090, 15), (1002417_9090, 16), (1002417_9090, 17), (1002417_9090, 18), (1002417_9090, 19), (1002417_9090, 20), (1002417_9090, 21), (1002417_9090, 22), (1002417_9090, 23), (1002417_9090, 24), (1002417_9090, 25), (1002417_9090, 26), (1002417_9090, 27), ...]

[179496 rows x 0 columns]

In [7]:
scaling_params = {}
scaling_params['output_stds'] = reformatted_outcomes_df.std().values
scaling_params['output_means'] = reformatted_outcomes_df.mean().values

In [8]:
ds = MIMIC3RealDataset(
    treatments=treatment_df,
    outcomes=reformatted_outcomes_df,
     vitals=pivoted_features_df,
     static_features=pd.DataFrame(None),
     outcomes_unscaled=reformatted_outcomes_df,
     scaling_params=scaling_params,
     subset_name='train'
)

In [9]:
ds.data

{'sequence_lengths': array([71, 71, 71, ..., 71, 71, 71]),
 'prev_treatments': array([[[4.],
         [4.],
         [4.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        [[7.],
         [7.],
         [7.],
         ...,
         [4.],
         [4.],
         [4.]],
 
        [[7.],
         [7.],
         [7.],
         ...,
         [7.],
         [7.],
         [7.]],
 
        ...,
 
        [[7.],
         [7.],
         [7.],
         ...,
         [3.],
         [3.],
         [3.]],
 
        [[7.],
         [7.],
         [7.],
         ...,
         [7.],
         [7.],
         [7.]],
 
        [[7.],
         [7.],
         [7.],
         ...,
         [7.],
         [7.],
         [7.]]]),
 'vitals': array([[[-0.30182825, -0.64252919, -0.2857695 , ...,  0.01302788,
           1.        ,  1.15325429],
         [-0.30182825, -0.64252919, -0.2857695 , ...,  0.01302788,
           1.        ,  1.15325429],
         [-0.30182825, -0.64252919, -0.2857695 

In [10]:
from src.data import RealDatasetCollection


class MIMIC3RealDatasetCollection(RealDatasetCollection):
    """
    Dataset collection (train_f, val_f, test_f)
    """
    def __init__(self,
                 ds: MIMIC3RealDataset,
                 projection_horizon: int = 5,
                 autoregressive=True,
                 **kwargs):
        """
        Args:
            path: Path with MIMIC-3 dataset (HDFStore)
            min_seq_length: Min sequence lenght in cohort
            max_seq_length: Max sequence lenght in cohort
            seed: Seed for random cohort patient selection
            max_number: Maximum number of patients in cohort
            split: Ratio of train / val / test split
            projection_horizon: Range of tau-step-ahead prediction (tau = projection_horizon + 1)
            autoregressive:
        """
        super(MIMIC3RealDatasetCollection, self).__init__()
        self.train_f = ds
        self.val_f = ds
        self.test_f = ds

        self.projection_horizon = projection_horizon
        self.has_vitals = True
        self.autoregressive = autoregressive
        self.processed_data_encoder = True

In [11]:
ds_collection = MIMIC3RealDatasetCollection(ds)

In [12]:
ds_collection.train_f.data

{'sequence_lengths': array([71, 71, 71, ..., 71, 71, 71]),
 'prev_treatments': array([[[4.],
         [4.],
         [4.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        [[7.],
         [7.],
         [7.],
         ...,
         [4.],
         [4.],
         [4.]],
 
        [[7.],
         [7.],
         [7.],
         ...,
         [7.],
         [7.],
         [7.]],
 
        ...,
 
        [[7.],
         [7.],
         [7.],
         ...,
         [3.],
         [3.],
         [3.]],
 
        [[7.],
         [7.],
         [7.],
         ...,
         [7.],
         [7.],
         [7.]],
 
        [[7.],
         [7.],
         [7.],
         ...,
         [7.],
         [7.],
         [7.]]]),
 'vitals': array([[[-0.30182825, -0.64252919, -0.2857695 , ...,  0.01302788,
           1.        ,  1.15325429],
         [-0.30182825, -0.64252919, -0.2857695 , ...,  0.01302788,
           1.        ,  1.15325429],
         [-0.30182825, -0.64252919, -0.2857695 